# Introduction to Partial Differential Equations
---

## Chapter 1: Preliminaries (Calculus, Linear Algebra, ODEs, and Python)
---

## Want to use Colab? [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp1/Chp1Sec3.ipynb)

---

## Prepping the environment for interactive plots in Colab
---

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab - installing missing packages')
    !pip install ipympl
    from IPython.display import clear_output
    clear_output()
    exit()
else:
    print('Not running on CoLab - assuming environment has necessary packages')

In [ ]:
%matplotlib widget
if 'google.colab' in str(get_ipython()):
    from google.colab import output
    output.enable_custom_widget_manager()

## Creative Commons License Information
---

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/80x15.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Introduction to Partial Differential Equations: Theory and Computations</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" property="cc:attributionName" rel="cc:attributionURL">Troy Butler</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" rel="dct:source">https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations</a>.

## Section 1.3: Numerical Estimation of an ODE
---

![Taylor series of exp](https://upload.wikimedia.org/wikipedia/commons/6/62/Exp_series.gif "Taylor polynomial approximations of the exponential function")

The most commonly used finite difference schemes are derived by manipulating [Taylor series expansions](https://en.wikipedia.org/wiki/Taylor_series) (the above gif shows various Taylor polynomial approximations to $e^x$ obtained by truncating its Taylor series expansion). This *usually* means it is straightforward to derive the [*local truncation error*](https://en.wikipedia.org/wiki/Truncation_error_(numerical_integration)) and prove convergence (which, for a finite difference scheme, usually means that we prove the scheme is both *consistent* and *stable*, which then implies convergence of the scheme). 

> Consistency is related to the truncation error and is usually straightforward to prove. Proving stability is usually an exercise in patience and perseverance. Stability concepts are discussed in the [next notebook](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp1/Chp1Sec4.ipynb).

We review the derivation of the forward Euler method in this notebook.

Some key takeaways are:

- The forward Euler method (or FE method for short) is an explicit method, which means it is easy to implement for problems where you want to evolve a solution in time quickly.

- However, the time steps required for numerical stability may be so small as to render the method useless for even simple problems. 

- The forward Euler method is often used as a *first attempt* at gaining some numerical insight into a solution that varies in time. 

- For improved stability and accuracy with larger time steps, you will often turn to either multistage explicit methods (e.g., the popular 4-stage Runge-Kutta method) or implicit methods (e.g., backward Euler or Crank-Nicolson). 

<mark>The best takeaway from this notebook is the way in which convergence is studied **numerically**</mark>. This is related to what is shown in the [previous notebook](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp1/Chp1Sec2.ipynb).

- Numerical studies of convergence are useful for either confirming the theory in practice, revealing some error we did not anticipate (either in the theoretical development or in the implementation), or for helping us develop the theory when the numerical analysis is a bit elusive. We develop some initial insight into this interplay of numerics and theory. 

---
### Section 1.3.1: Recalling [Taylor Series](https://en.wikipedia.org/wiki/Taylor_series) and [Taylor's Theorem](https://en.wikipedia.org/wiki/Taylor%27s_theorem)
---

For a *smooth* (i.e., infinitely differentiable) function $g(t)$, its Taylor series expanded about $t=a$ is given by

$$
\begin{align}
    g(t) &= g(a)+\frac {g'(a)}{1!} (t-a)+ \frac{g''(a)}{2!} (t-a)^2+\frac{g'''(a)}{3!}(t-a)^3+ \cdots \\\\
     & = \sum_{n=0} ^ {\infty} \frac {g^{(n)}(a)}{n!} (t-a)^{n}.
\end{align}
$$

When $a=0$, this is often referred to as the Maclaurin series (but we'll stick with referring to this as a Taylor series). 

**Remarks:**

- The equality of the infinite series to $g(t)$ is understood to mean that the limit of the sequence of partial sums (i.e., the Taylor polynomials $T_k(t) := \sum_{n=0} ^ {k} \frac {g^{(n)}(a)}{n!} (t-a)^{n}$) converges pointwise to $g(t)$as $k\to \infty$ on some interval (which may even be $\mathbb{R}$), and the convergence is uniform on every compact subset (i.e., sets of the form $[c,d]$ for some real numbers $c<d$) of the convergence interval.

  - Recall from the previous notebook that uniform convergence means convergence in the $\sup$-norm metric.

Of course, we *never* in practice construct or compute infinite series. Who would engage in such madness? In practice, we make do with a particular Taylor polynomial (i.e., some truncation of a Taylor series defining a polynomial of a specific degree) and utilize Taylor's theorem to understand the error. 

---
#### Taylor's Theorem

Let $k\geq 1$ be an integer and let the function $g:\mathbb{R}\to\mathbb{R}$ be $k$ times differentiable at the point $a\in\mathbb{R}$. Then, there exists a function $h_k:\mathbb{R}\to\mathbb{R}$ such that

$$
    g(t) = T_k(t) + h_k(t)(t-a)^k, \ \text{ and } \ \lim_{t\to a}h_k(x)=0.
$$

---

**Remarks:**

- The polynomial $T_k(t)$ appearing above is the $k$th order Taylor polynomial we defined above. 

- The remainder term is
<br><br>
$$
    R_k(t) = g(t) - T_k(t) =  h_k(t)(t-a)^k = o(|t-a|^k) \ \text{ as } t\to a.
$$
<br><br>
Here, $R_k(t) = o(|t-a|^k)$ as $t\to a$ means that $\lim_{t\to a} \left|\dfrac{R_k(t)}{|t-a|^k}\right| = 0$. 
<br><br>
Thus, Taylor's theorem describes the polynomial (given by $T_k$) that is the *asymptotic best fit* to the function $g(t)$ as $t\to a$.

- Note that this theorem *fails* to give an actual form for the remainder term $R_k(t)$ because we do not know the form of $h_k(t)$. The theorem only states there exists such a function $h_k(t)$ but does not state what it actually is other than it must have a specific asymptotic property. This is unsatisfying.

- <mark>The real power of approximating a function with a Taylor polynomial where an explicit remainder term can be formulated and analyzed occurs when we use stronger *regularity* assumptions on the function.</mark>

Wait, what does regularity mean? In PDEs, we often refer to the regularity of the function when discussing just *how smooth* the function is. If we assume that just *one more derivative* of $g$ exists so that it is $k+1$ times differentiable on the open interval defined by $a$ and $t$ and that the $k$th derivative is continuous on the closed interval defined by $a$ and $t$, then the remainder term $R_k(t)$ takes the form

$$
    R_k(t) = \frac { g^{(k+1)} (\xi) } {(k+1)!} (t-a)^{k+1}, 
$$

where $\xi$ is some number between $a$ and $t$ that is guaranteed to exist by the Mean Value Theorem for derivatives.

In practice, we do not know what $\xi$ is, but if we can determine a bound $g^{(k+1)}(\xi)$ on the interval defined by $(a-r, a+r)$ for some $r>0$ (and call such a bound $M$), then we can bound the remainder term as follows

$$
    \left| R_k(t) \right| \leq \frac{M r^{k+1}}{(k+1)!}, 
$$

This then defines a computable bound on the error for the Taylor polynomial for any $t\in(a-r, a+r)$. Moreover, the bound is $\mathcal{O}(r^{k+1})$ which means that $R_k\sim \mathcal{O}(r^{k+1})$. Subsequently, $\frac{R_k}{r^m}\sim\mathcal{O}(r^{k+1-m})$ for $1\leq m\leq k$, which is useful for understanding the rates of convergence we will encounter with finite difference schemes.

Let's look at a simple example involving $e^t$. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import factorial as fac

In [ ]:
# Defining the Taylor polynomial for e^t

def exponential_Taylor(t, a, k):
    v = np.exp(a) * np.ones(len(t))
    for i in range(1, k+1):
        v += np.exp(a) * (t-a)**i / fac(i)
    return v

# Define the remainder bound
def exponential_remainder(t, a, k):
    r = np.max(np.abs(t-a))
    R_k = np.exp(np.max(t)) * r**(k+1) / fac(k+1)
    return R_k

In [ ]:
def plot_exponential(t, a, k, num=0):
    plt.figure(num = num)
    plt.clf()
    
    plt.subplot(1,2,1)
    plt.plot(t, np.exp(t), 'b', label='$e^t$')
    plt.plot(t, exponential_Taylor(t, a, k), 'r:', label='$T_' + str(k) + '(t)$')
    plt.legend()
    
    plt.subplot(1,2,2)
    plt.plot(t, exponential_remainder(t, a, k)*np.ones(len(t)), 'b', label='Remainder Bound')
    plt.plot(t, np.abs(np.exp(t) - exponential_Taylor(t, a, k)), 'r:', label='abs. error = $|e^t - T_' + str(k) + '(t)|$')
    plt.legend()
    plt.show()

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
%reset -f out 

%matplotlib widget
interact_manual(plot_exponential, 
            t = fixed(np.linspace(-1, 1, 50)),
            a = fixed(-0.5),  # Try -0.5, 0, and 0.5
            k = widgets.IntSlider(value=1, min=1, max=10, step=1),
            num = fixed(0))

---
### Section 1.3.2: A prototypical IVP and the Forward Euler Method
---

In the remainder (no pun intended) of this notebook, we consider 1st order IVPs of the form

$$
    \begin{cases}
        u'(t) &= f(t, u(t)), \ t>t_0 \\
        u(t_0) &= u_0,
    \end{cases}
$$

where $f=f(t, u)$ is a given function that may result in the IVP being nonlinear. 

For simplicity, we assume $t_0=0$ in everything that follows in this notebook.

While we provide a detailed derivation and analysis of the forward Euler method below, conceptually, it is rather simple to describe as replacing $u'(t)$ with a first-order forward finite difference approximation at any given time, keeping the right-hand side evaluated at the current time $t$, and solving for $u(t+\Delta t)$ (where $\Delta t$ is used to denote the step-size in the finite difference formula). 

---
#### Deriving the [Forward Euler Method](https://en.wikipedia.org/wiki/Euler_method)
---

Suppose $u$ is twice continuously differentiable around $t$ and that $\Delta t>0$ is chosen such that $t+\Delta t$ is "around" $t$, then by Taylor's theorem and the explicit representation of the remainder term, we have

$$
    u(t+\Delta t) = u(t) + u'(t)\Delta t + \frac{u''(\xi)(\Delta t)^2}{2}
$$

for some $\xi\in[t, t+\Delta t]$. Some people prefer to write this as

$$
    u(t+\Delta t) = u(t) + u'(t)\Delta t + \frac{u''(t+\xi)(\Delta t)^2}{2}
$$

where now $\xi\in[0,\Delta t]$.

By rearranging terms, we have that

$$
\begin{align}
    u'(t) &= \frac{u(t+\Delta t) - u(t)}{\Delta t} +  \frac{u''(t+\xi)\Delta t}{2} \\\\
          &= \frac{u(t+\Delta t) - u(t)}{\Delta t} + \mathcal{O}(\Delta t).
\end{align}
$$

Let $t_m:= m\Delta t$ and $v_m$ denote the approximations we will construct to $u(t_m)$ for $m=0, 1, 2, \ldots$. 

At $m=0$, we choose $v_0=u(t_0)=u_0$ to produce an approximation with zero error.

What should we do for $m>0$?

Well, let's explore what we know about $u(t)$. We know it satisfies the IVP, and we know that $\dfrac{u(t+\Delta t) - u(t)}{\Delta t}$ approximates $u'(t)$ for any $t$, so we have that

$$
    \frac{u(t_{m+1}) - u(t_m)}{\Delta t} \approx u'(t_m) = f(t_m, u(t_m))
$$

is a "good" approximation if $\Delta t$ is sufficiently small. This seems like a decent way of recursively defining our approximations to $u(t_m)$ with $v_m$ by simply substituting the $v_m$ into this approximation of the differential equation to get

$$
    \frac{v_{m+1}-v_m}{\Delta t} = f(t_m, v_m), 
$$

which we can solve for $v_{m+1}$ to get

$$
    v_{m+1} = v_m + \Delta t f(t_m, v_m).
$$

Thus, as soon as we have $v_0$, we can compute $v_1$, from which we can compute $v_2$, and so on to produce a sequence of $v_m$ values that approximate $u(t_m)$ for any $m=0,1,2,\ldots$. 

This is called the Forward Euler method. 

**Remarks:**

- This method sucks. Well, it is okay. Nah, it sucks. It is easy to implement and straightforward to analyze, but overall it is not great even on simple problems. It is usually our first attempt just to get some initial transitory information on how a state variable evolves over very short time periods, but we typically prefer to use other methods.

- While we have derived an approximation method above, we have not yet analyzed whether it is any good or not. We just used some reasoning/rationalization to justify some substitutions that "seemed reasonable" as we went along. This is fine for developing a method based on some mathematical reasoning/intuition, but we still need to perform some sort of rigorous analysis of this approach.

First, let's explore some examples.

In [ ]:
def forward_Euler(f, Delta_t, n, u_0):
    # Assuming f is passed as a function of (t,u)
    v = np.zeros(n)
    v[0] = u_0
    for i in range(1,n):
        v[i] = v[i-1] + Delta_t * f(i*Delta_t, v[i-1])
    return v

In [ ]:
# A "trivial" example for u'=u, so f(t,u)=u
f = lambda t, u: u  

u_0 = 1

u = lambda t: u_0*np.exp(t)

In [ ]:
def plot_forward_Euler(f, u, t_f, Delta_t, num=1):
    # Assuming f is a function of (u,t) and u is a function of t
    
    plt.figure(num)
    plt.clf()
    
    n = int(t_f/Delta_t)
    t = np.linspace(0, t_f, n+1)
    
    plt.plot(t, forward_Euler(f, t[1]-t[0], n+1, u(0)), 'r:', 
             marker='s', label='Approx. soln')
    
    plt.plot(t, u(t), 'b', label='Exact soln')
    
    plt.legend()
    plt.show()

In [ ]:
%reset -f out 

%matplotlib widget
interact_manual(plot_forward_Euler, 
            f = fixed(f),
            u = fixed(u),
            t_f = fixed(1),
            Delta_t = widgets.FloatText(value=0.1),
            num = fixed(1))

In [ ]:
# A nonlinear example
f = lambda t, u: t*u*(u-2)

u_0 = 1.99

u = lambda t: 2*u_0 / (u_0+(2-u_0)*np.exp(t**2))

In [ ]:
%reset -f out 

%matplotlib widget
interact_manual(plot_forward_Euler, 
            f = fixed(f),
            u = fixed(u),
            t_f = widgets.FloatText(4),
            Delta_t = widgets.FloatText(value=0.1),  # Try 0.25, 0.5, and 1
            num = fixed(2))

In [ ]:
# A nonlinear example
f = lambda t, u: t*u*(u-2)

u_0 = 2.01

u = lambda t: 2*u_0 / (u_0+(2-u_0)*np.exp(t**2))

In [ ]:
%reset -f out 

%matplotlib widget
interact_manual(plot_forward_Euler, 
            f = fixed(f),
            u = fixed(u),
            t_f = widgets.FloatText(2.5),
            Delta_t = widgets.FloatText(value=0.1),  # Try 0.01 a
            num = fixed(3))

**What if we did not necessarily know what $u$ is, so we wanted to improve the `plot_forward_Euler` function to allow for the option of $u$ being unknown?**

In [ ]:
def plot_forward_Euler_improved(f, t_f, Delta_t, u_0, u=None, num=1):  # All keyworded parameters have to come at the end
    # Assuming f is a function of (u,t) and u is a function of t
    
    plt.figure(num)
    plt.clf()
    
    n = int(t_f/Delta_t)
    t = np.linspace(0, t_f, n+1)
    
    plt.plot(t, forward_Euler(f, t[1]-t[0], n+1, u_0), 'r:', 
             marker='s', label='Approx. soln')
    
    if u is not None:
        plt.plot(t, u(t), 'b', label='Exact soln')
    
    plt.legend()
    plt.show()

In [ ]:
%reset -f out 

%matplotlib widget
interact_manual(plot_forward_Euler_improved, 
            f = fixed(lambda t, u: u*np.sin(u*t)),  # Put any f you want here whether you know u or not. 
            u = fixed(None),  # Who cares what the exact solution $u$ is? Not me!
            u_0 = fixed(1),  # Put any IC you want here whether you know $u$ or not. 
            t_f = widgets.FloatText(5),
            Delta_t = widgets.FloatText(value=0.1),  # Try 0.01
            num = fixed(3))

---
### Section 1.3.3: An initial error analysis and ROC analysis
---

So far, we have only explored the quality of the FE method in a qualitative sense by examining how the plots compare between the approximations $\{v_m\}$ and the exact solutions.

We now explore the errors a bit more systematically.

Returning to the trivial example of $u'(t)=u(t)$ with $u(0)=1$ that was numerically explored above, we see that since $f(t,u(t))=u(t)$ that the FE method produces the following formula for the $v_m$ approximations

$$
\begin{align}
    v_{m+1} &= v_m + \Delta t v_m \\\\
            &= (1+\Delta t) v_m.
\end{align}
$$

Using induction and the fact that $v_0=u(0)=1$, we have that for any $m=0, 1, 2,\ldots$, 

$$
    v_m = (1+\Delta t)^m.
$$

Suppose we want to study the error at a final time $t_f$. For the sake of simplicity, we assume $t_f$ is a multiple of $\Delta_t$, i.e., $t_f=M\Delta t$ for some $M\in\mathbb{N}$. Then, we are interested in 

$$
\begin{align}
    v_M &= (1+\Delta t)^M \\\\
        &= (1+\Delta t)^{t_f/\Delta t}.
\end{align}
$$

The actual solution $u(t_f)=e^{t_f}$, so the error in $v_M$ as a function of $\Delta t$ is given by

$$
    E(\Delta t) = \left| e^{t_f} - (1+\Delta t)^{t_f/\Delta t} \right|.
$$

The question becomes: Is $(1+\Delta t)^{t_f/\Delta t}$ ever a good approximation to $e^{t_f}$ for "small" values of $\Delta t$? 

To answer this question, we first recall from calculus the [limit characterization of the exponential function](https://en.wikipedia.org/wiki/Characterizations_of_the_exponential_function) (which is something we also saw in the previous notebook):

$$
    \lim_{n\to\infty} \left(1+\frac{x}{n}\right)^n = e^x, 
$$

which we can use a change of variables of $\epsilon=x/n$ (so $n=x/\epsilon$) to rewrite as

$$
    \lim_{\epsilon\to 0} \left(1+\epsilon\right)^{x/\epsilon} = e^x.
$$

Now, we see for small $\Delta t$ that $(1+\Delta t)^{t_f/\Delta t}\approx e^{t_f}$, and in fact that

$$
    E(\Delta t) \to 0 \text{ as } \Delta t\to 0.
$$

Let's explore this below.

In [ ]:
# Create an array of Delta_ts that decrease by an order of magnitude

Delta_ts = np.logspace(-1, -7, 7)
print(Delta_ts)

In [ ]:
t_f = 1  # can choose this to be any positive number

abs_errors = np.abs(np.exp(t_f)-(1+Delta_ts)**(t_f/Delta_ts))
print(abs_errors)

In [ ]:
%matplotlib widget

plt.figure(4)
plt.loglog(Delta_ts, abs_errors, 'b')

line_params = np.polyfit(np.log(Delta_ts), np.log(abs_errors), 1)

plt.title(r'Rate of Convergence of $E(\Delta t)\approx${:1.3f}'.format(line_params[0]))

**Exploring the ROC for the nonlinear example.**

In [ ]:
f = lambda t, u: t*u*(u-2)

u_0 = 2.01

u = lambda t: 2*u_0 / (u_0+(2-u_0)*np.exp(t**2))

In [ ]:
Delta_ts = np.logspace(-1, -7, 7)
print(Delta_ts)

In [ ]:
# This code cell may take a few moments to run

t_f = 1  # can choose this to be any positive number

abs_errors = np.zeros(len(Delta_ts))
for i, Delta_t in zip(range(len(Delta_ts)), Delta_ts):
    abs_errors[i] = np.abs(u(t_f)-forward_Euler(f, Delta_t, int(1/Delta_t)+1, u_0)[-1])
print(abs_errors)

In [ ]:
%matplotlib widget

plt.figure(4)
plt.loglog(Delta_ts, abs_errors, 'b')

line_params = np.polyfit(np.log(Delta_ts), np.log(abs_errors), 1)

plt.title(r'Rate of Convergence of $E(\Delta t)\approx${:1.3f}'.format(line_params[0]))

---
### Section 1.3.4: Exploring the errors in more depth
---

<mark>**Step 1: Establish the Local Truncation Error (LTE)**</mark>

Let $\tau_m$ denote the Local Truncation Error (LTE) of the FE method at one-step of the method using the *exact* solution. From the above derivation of the FE method, we see that 

$$
    \large \tau_m = \frac{1}{2}(\Delta t) u''(\xi)
$$

where $\xi\in(t_m,t_{m+1})$.

Generally, we will seek a ***bound on $u''(\xi)$ over $[0,t_f]$ in order to remove dependency of $\tau_m$ on $m$*** (notice that the dependence is subtle because $\xi\in(t_m,t_{m+1})$. How do we do this? Well, $u'=f(t,u)$ which means that $u''$ can be written in terms of derivatives of $f$ and $f$ is known. Of course, if $f$ is not differentiable, then we are not going to have much luck proving convergence results or at least not proving rates of convergence.

Since in this particular case we know that $u(t)=e^t$ is the exact solution to this simple IVP, we have that $u''(t)=e^t$ so that $u''(\xi)=e^\xi$, which implies 

$$
    \large \tau_m = \frac{\Delta t}{2}e^\xi.
$$

Since $e^t$ is a monotonically increasing function, we have that $e^\xi\leq e^{t_f}$, which implies that

$$
    \large |\tau_m| \leq \frac{\Delta t}{2} e^{t_f}, \ \text{ for } \ 0\leq (m+1)\Delta t\leq t_f.
$$

Here, we wrote $0\leq (m+1)\Delta t\leq t_f$ because the truncation error $\tau_m$ is defined for the method stepping from $t_m$ to $t_{m+1}$.

<mark>**Step 2: Determine a formula for how the error evolves over time**</mark>

Write $E_m=u_m-v_m$ to denote the error at time $t_m=m\Delta t$, then 

$$
    \large u_{m+1} = u_m + \Delta t u_m + \Delta t \tau_m = (1+\Delta t)u_m + \Delta t\tau_m.
$$

So,

$$
    \large E_{m+1}=u_{m+1}-v_{m+1} = (1+\Delta t)u_m + \Delta t \tau_m - (1+\Delta t)v_m. 
$$

By factoring, we have that

$$
    \large E_{m+1} = (1+\Delta t)E_m + \Delta t \tau_m. 
$$

Since $u_0 = 1 = v_0$, we clearly have $E_0=0$.

Why did we use $E_m$ for the error term instead of $e_m$ as in the previous notebook? Well, $e$ is being used here for the exponential function, and we want to avoid "overloading" the notation $e$ in this current notebook.

<mark>**Step 3: Use induction to determine a bound for $E_m$ at each $m$**</mark>

It is often helpful to determine the first few bounds before applying induction.

For $m=1$, we have that 

$$
    \large |E_1| \leq \Delta t \tau_1 \leq \frac{(\Delta t)^2}{2} e^{t_f}
$$

so for $m=2$, we have that

$$
    \large |E_2| \leq (1+\Delta t) \frac{(\Delta t)^2}{2} e^{t_f}+ \frac{(\Delta t)^2}{2} e^{t_f}.
$$

Then, for $m=3$, we have that

$$
    \large |E_3| \leq (1+\Delta t)^2 \frac{(\Delta t)^2}{2} e^{t_f} + (1+\Delta t)\frac{(\Delta t)^2}{2} e^{t_f} + \frac{(\Delta t)^2}{2} e^{t_f}.
$$

We then see from induction that

$$
    \large |E_m| \leq \frac{(\Delta t)^2}{2}e^{t_f} \sum_{i=0}^{m-1} (1+\Delta t)^i.
$$

The sum is a partial sum of a geometric series, which can easily be calculated to show that

$$
    \large |E_m| \leq \frac{(\Delta t)^2}{2} e^{t_f} \left[\frac{(1+\Delta t)^m - 1}{\Delta t}\right], 
$$

which gives

$$
    \large |E_m| \leq \frac{\Delta t}{2} e^{t_f} \left[(1+\Delta t)^m - 1\right].
$$

We now use the inequality $1+\Delta t\leq e^{\Delta t}$ and observe that $(e^{\Delta t})^m = e^{m\Delta t} = e^{t_m}$ to write this as

$$
    \large |E_m| \leq \frac{\Delta t}{2} e^{t_f}\left(e^{t_m}-1\right).
$$

**Remarks:**

- If we compare this to our computed errors from above (which we do below), we notice this bound is too large. Well, it is a *bound* not an estimate, so keep that in mind.

- When we do not know the solution, which is often the case in practice, we would replace the $e^{t_f}$ in the bound for $|E_m|$ with $\sup_{0\leq t\leq t_f} |u''(t)|$ and use the relationship of $u''(t)=\frac{d}{dt} f(t,u(t)) = (\partial_t f) + (\partial_u f)u' = (\partial_t f) + (\partial_u f)f$ to try and get an idea of the size of the second derivative, which may only allow us to make local approximations to the bound of error at each time step instead of a global bound that is useful over all time steps.

- If we replace the $e^{t_f}$ with $\sup_{0\leq t\leq t_f} |u''(t)|$, then this is also a bound derived *a priori* of the computed solution. We simply used the form of the problem to get at this error bound. It is *common* for a priori derived error bounds and estimates of these error bounds to be *orders of magnitude* too large.

- If an error bound can be orders of magnitude too large, then is it good for anything? Well, yes, it is good for proving convergence and establishing rates of convergence (e.g., under the assumption that the second derivative of $u$ is bounded on $[0, t_f]$, then <mark>we have that the method is $\mathcal{O}(\Delta t)$</mark> according to the error bound). But, a priori error bounds are just bad for producing any kind of reliable estimate of the actual error. This is not what their purpose is though. A posteriori error estimation is really the desired tool for producing accurate estimates of errors in computed quantities from a numerical solution, but this is a topic that is best studied in more depth in an advanced PDEs course where we can apply some functional analysis tools.

In [ ]:
# Create an array of Delta_ts that decrease by an order of magnitude

Delta_ts = np.logspace(-1, -7, 7)
print(Delta_ts)

In [ ]:
t_f = 1  # can choose this to be any positive number

abs_errors = np.abs(np.exp(t_f)-(1+Delta_ts)**(t_f/Delta_ts))
print('Absolute value of errors at time t_f\n')
print(abs_errors)

print('-'*50)

error_bds = Delta_ts/2*np.exp(t_f)*(np.exp(t_f)-1)
print('\nError bound at time t_f\n')
print(error_bds)

print('-'*50)
print('\nRatio of error bound to actual magnitude of errors\n')
print(error_bds/abs_errors)

---
## Navigation:

- [Previous](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp1/Chp1Sec2.ipynb)

- [Next](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp1/Chp1Sec4.ipynb)
---